<a href="https://colab.research.google.com/github/dudumlc/ML_Classification/blob/main/Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. IMPORTAÇÃO DAS BIBLIOTECAS E DO DATASET A SER ESTUDADO

In [40]:
# IMPORTAÇÃO DAS BIBLIOTECAS A SEREM USADAS

import pandas as pd
import numpy as np
from pandas.api.types import infer_dtype

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from sklearn.model_selection import GridSearchCV

In [41]:
# IMPORT DO DATASET
df_raw = pd.read_csv('churn.csv')

> Foi decidido criar uma cópia do dataset em outra variável para manter o dataframe bruto (df_raw) intacto. Dessa forma, caso alguma alteração seja feita erroneamente na variável cópia, o dataset original não precisará ser importado novamente, pois ele estará salvo como backup.

In [42]:
# CRIANDO UMA CÓPIA DO DATASET ORIGINAL
df = df_raw.copy()

## 2. ENTENDIMENTO DO DATASET

In [43]:
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [44]:
# DIMENSÕES DO DATASET
df.shape

(7043, 21)

In [45]:
# QUANTIDADE DE LINHAS DUPLICADAS
df.duplicated().sum()

0

In [46]:
# QUANTIDADE DE DADOS NULOS POR COLUNA
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [47]:
# TIPO DE DADOS DE CADA COLUNA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [48]:
# QUANTIDADE DE CADA TIPO DE DADOS
df.dtypes.value_counts()

object     18
int64       2
float64     1
dtype: int64

> A análise da granularidade das colunas é feita já previamente para saber quais colunas focar na etapa de limpeza dos dados, pois colunas com granularidade muito alta (muitos dados distintos) não são muito informativas para o modelo e, por isso, devem ser analisadas para decidir a real necessidade de serem mantidas. No caso abaixo, as colunas muito granulares são as numéricas (o que é comum, já que cada mínima diferença entre os números representa um novo dado distinto) e a coluna ID, que representa um código diferente por cliente. Ela não é informativa e deveria ser retirada do modelo, porém será mantida na relação apenas para que, ao final do projeto, seja possível colocá-la ao lado da respectiva predição e assim, poder associar o cliente ao churn respectivo predito.

In [49]:
# GRANULARIDADE DE CADA COLUNA
df.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

## 3. SEPARAÇÃO EM MASSA DE TREINO E TESTE
#### Separação sendo feita antes do pré-processamento dos dados para evitar vazamento (Data Leakage)

In [50]:
# CRIAÇÃO DA VARIÁVEL TARGET, FEATURES E SEPARANDO A COLUNA ID
ID = df.iloc[:,0]
X = df.iloc[:,1:-1].copy()
y = df['Churn'].replace({'Yes':1,'No':0}).copy()

In [51]:
# SEPARAÇÃO DAS MASSAS DE TREINO, VALIDAÇÃO E TESTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## 4. LIMPEZA E PRÉ-PROCESSAMENTO DOS DADOS

> Foi percebido que haviam 11 células da coluna TotalCharges com apenas um espaço (' ') escrito.

In [52]:
# QUANTIDAED DE CÉLULAS QUE POSSUEM APENAS UM ESPAÇO (' ') NA COLUNA TotalCharges
print('Quantidade de células preenchidas por um espaço no df de treino:',    len(X_train[X_train['TotalCharges'] == ' ']))
print('Quantidade de células preenchidas por um espaço no df de teste:',     len(X_test[X_test['TotalCharges'] == ' ']))

Quantidade de células preenchidas por um espaço no df de treino: 8
Quantidade de células preenchidas por um espaço no df de teste: 3


> Para não perder as informações das outras features preenchidas, foi decidido não apagar a linha e inputar a média da coluna no lugar dessas células. Contudo, para conseguir extrair a média é necessário alterar a coluna para o tipo *Float*, e para isso é necessário remover a string que estava escrita indevidamente (' '). Com isso, substituirei essas células por um valor nulo e posteriormente substuirei os valores nulos pela média calculada dos valores da coluna.

In [53]:
# SUBSTITUIÇÃO DAS CÉLULAS " " POR UM VALOR NULO
X_train['TotalCharges'] = X_train['TotalCharges'].replace(' ',None)
X_test['TotalCharges'] = X_test['TotalCharges'].replace(' ',None)

In [54]:
# ALTERANDO A COLUNA TotalCharges PARA TIPO FLOAT
X_train['TotalCharges'] = X_train['TotalCharges'].astype('float')
X_test['TotalCharges'] = X_test['TotalCharges'].astype('float')

In [55]:
# FINALMENTE, SUBSTITUINDO AS CÉLULAS PELA MÉDIA DA COLUNA
X_train['TotalCharges'] = X_train['TotalCharges'].fillna(X_train['TotalCharges'].mean())
X_test['TotalCharges'] = X_test['TotalCharges'].fillna(X_test['TotalCharges'].mean())

> Os tipos de dados das outras colunas também serão ajustados. Dessa forma, será garantido que as colunas numéricas estarão em formato de *Float* ou *Int* e que as colunas categóricas estarão em formato de *Category* (é mais correto e economiza mais espaço do armazenamento do que manter colunas no formato *Object*).

In [56]:
# ALTERANDO COLUNAS DO DATASET DE TREINO PARA CATEGORY
object_cols = X_train.select_dtypes(include='object').columns
X_train[object_cols] = X_train[object_cols].astype('category')

# ALTERANDO COLUNAS DO DATASET DE TESTE PARA CATEGORY
object_cols2 = X_test.select_dtypes(include='object').columns
X_test[object_cols2] = X_test[object_cols2].astype('category')

> O modo mais adequado e eficiente de estruturar os dados para treinar o modelo de Machine Learning é armazenar os dados em formato numérico. Isso porque a interpretabilidade e compatibilidade com os modelos é maior com esse tipo de dados. Dessa forma, será usada a técnica de OneHotEncoder para transformar as colunas categóricas em colunas numéricas.

In [57]:
print(f"São {len(X_train.columns.values)} colunas, sendo elas:",X_train.columns.values)

São 19 colunas, sendo elas: ['gender' 'SeniorCitizen' 'Partner' 'Dependents' 'tenure' 'PhoneService'
 'MultipleLines' 'InternetService' 'OnlineSecurity' 'OnlineBackup'
 'DeviceProtection' 'TechSupport' 'StreamingTV' 'StreamingMovies'
 'Contract' 'PaperlessBilling' 'PaymentMethod' 'MonthlyCharges'
 'TotalCharges']


In [58]:
# CRIANDO UMA VARIÁVEL PARA O ONEHOTENCODER. O PARÂMETRO DROP='IF-BINARY' EXCLUIRÁ AS COLUNAS BINÁRIAS
encoder = OneHotEncoder(drop='if_binary')
encoder

OneHotEncoder(drop='if_binary')

> Como apenas as features categóricas serão processadas pelo OneHotEncoder, criarei um dataframe com as colunas numéricas para que assim, após a conversão das colunas categóricas, seja possível unir os dados em um único df.

In [59]:
# CRIANDO UM DATAFRAME APENAS COM COLUNAS NUMÉRICAS
num_cols_train = X_train[X_train.select_dtypes(include='number').columns]
num_cols_test = X_test[X_test.select_dtypes(include='number').columns]

> O encoder será ajustado aos dados de treino (para evitar data leakage, caso fosse ajustado aos dados de teste também) e apenas transformará as outras bases

In [60]:
# LISTANDO AS COLUNAS CATEGÓRICAS - *não importa de qual base pegar. Teste, treino e validação possuem as mesmas colunas*
cat_cols = X_train.select_dtypes(exclude='number').columns

# AJUSTANDO O ENCODER AOS DADOS DE TREINO E APENAS TRANSFORMANDO OS OUTROS DADOS
encoder.fit(X_train[cat_cols])

transformed_cat_cols_train = encoder.transform(X_train[cat_cols]).toarray()
transformed_cat_cols_test = encoder.transform(X_test[cat_cols]).toarray()

> Com a matriz resultante do OneHotEncoder criada, é necessário convertê-la novmamente para dataframe e unir novamente os dados transformados com os dados numéricos do dataset.

In [61]:
# LISTANDO OS NOVOS NOMES DAS COLUNAS TRANSFORMADAS
transformed_cat_cols_names = encoder.get_feature_names_out(cat_cols)
print(f"Tornaram-se {len(transformed_cat_cols_names)} colunas, sendo elas:",transformed_cat_cols_names)

Tornaram-se 36 colunas, sendo elas: ['gender_Male' 'Partner_Yes' 'Dependents_Yes' 'PhoneService_Yes'
 'MultipleLines_No' 'MultipleLines_No phone service' 'MultipleLines_Yes'
 'InternetService_DSL' 'InternetService_Fiber optic' 'InternetService_No'
 'OnlineSecurity_No' 'OnlineSecurity_No internet service'
 'OnlineSecurity_Yes' 'OnlineBackup_No' 'OnlineBackup_No internet service'
 'OnlineBackup_Yes' 'DeviceProtection_No'
 'DeviceProtection_No internet service' 'DeviceProtection_Yes'
 'TechSupport_No' 'TechSupport_No internet service' 'TechSupport_Yes'
 'StreamingTV_No' 'StreamingTV_No internet service' 'StreamingTV_Yes'
 'StreamingMovies_No' 'StreamingMovies_No internet service'
 'StreamingMovies_Yes' 'Contract_Month-to-month' 'Contract_One year'
 'Contract_Two year' 'PaperlessBilling_Yes'
 'PaymentMethod_Bank transfer (automatic)'
 'PaymentMethod_Credit card (automatic)' 'PaymentMethod_Electronic check'
 'PaymentMethod_Mailed check']


In [62]:
# TRANSFORMANDO A MATRIZ EM DATAFRAME NOVAMENTE
transformed_cat_cols_train = pd.DataFrame(transformed_cat_cols_train, columns=transformed_cat_cols_names, index=X_train.index)
transformed_cat_cols_test = pd.DataFrame(transformed_cat_cols_test, columns=transformed_cat_cols_names, index=X_test.index)

In [63]:
# UNINDO O DATAFRAME CRIADO COM O DATAFRAME QUE ARMAZENOU AS VARIÁVEIS NUMÉRICAS
X_train_final = pd.concat([transformed_cat_cols_train,num_cols_train],axis=1)
X_test_final = pd.concat([transformed_cat_cols_test,num_cols_test],axis=1)

In [64]:
# QUANTIDADE DE COLUNAS DO DATASET FINAL COM AS COLUNAS CATEGÓRICAS TRATADAS
print(f"O processo do OneHotEncoder retornou {X_train_final.shape[1]} colunas.")

O processo do OneHotEncoder retornou 40 colunas.


In [65]:
# ANALISANDO COLUNAS RESULTANTES
X_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4930 entries, 6268 to 5966
Data columns (total 40 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   gender_Male                              4930 non-null   float64
 1   Partner_Yes                              4930 non-null   float64
 2   Dependents_Yes                           4930 non-null   float64
 3   PhoneService_Yes                         4930 non-null   float64
 4   MultipleLines_No                         4930 non-null   float64
 5   MultipleLines_No phone service           4930 non-null   float64
 6   MultipleLines_Yes                        4930 non-null   float64
 7   InternetService_DSL                      4930 non-null   float64
 8   InternetService_Fiber optic              4930 non-null   float64
 9   InternetService_No                       4930 non-null   float64
 10  OnlineSecurity_No                        4930

> É possível notar que há várias colunas redundantes. Por exemplo, a coluna 'PhoneService_Yes' e a coluna 'MultipleLines_No phone service' apresentam a mesma informação sobre a ausência de serviços telefônicos. Da mesma forma, várias colunas apresentam a mesma informação da coluna 'InternetService_No' sobre a ausência de serviços de internet. Dessa forma, excluirei essas colunas para diminuir a multicolinearidade das variáveis, a dimensionalidade da base de dados e, eventualmente, um overfitting causado pela redundância dos dados.

In [66]:
# CRIANDO UMA LISTA COM AS COLUNAS A SEREM RETIRADAS
cols_redundantes = []
for i in X_train_final.columns:
  if "No internet service" in i:
    cols_redundantes.append(i)
  elif "No phone service" in i:
    cols_redundantes.append(i)
  else:
    continue

> Além das colunas anteriores, também é possível notar colunas complementares uma com a outra. Por exemplo, as colunas "MultipleLines_No" e "MultipleLines_Yes" são complementares. Dessa forma, uma delas pode ser excluída, visto que a informação de uma coluna está implícita na outra (quando "MultipleLines_No" for 1, consequentemente "MultipleLine_Yes" vai ser 0).

In [67]:
cols_redundantes

['MultipleLines_No phone service',
 'OnlineSecurity_No internet service',
 'OnlineBackup_No internet service',
 'DeviceProtection_No internet service',
 'TechSupport_No internet service',
 'StreamingTV_No internet service',
 'StreamingMovies_No internet service']

In [80]:
# INCLUINDO AS COLUNAS COMPLEMENTARES NA LISTA DE COLUNAS REDUNDANTES
for i in X_train_final.columns:
  if '_No' in i:
    cols_redundantes.append(i)
  else:
    continue

In [82]:
# RETIRANDO AS COLUNAS DE TODAS AS BASES
X_train_final.drop(cols_redundantes, axis=1,inplace=True)
X_test_final.drop(cols_redundantes, axis=1,inplace=True)

In [83]:
X_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4930 entries, 6268 to 5966
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   gender_Male                              4930 non-null   float64
 1   Partner_Yes                              4930 non-null   float64
 2   Dependents_Yes                           4930 non-null   float64
 3   PhoneService_Yes                         4930 non-null   float64
 4   MultipleLines_Yes                        4930 non-null   float64
 5   InternetService_DSL                      4930 non-null   float64
 6   InternetService_Fiber optic              4930 non-null   float64
 7   OnlineSecurity_Yes                       4930 non-null   float64
 8   OnlineBackup_Yes                         4930 non-null   float64
 9   DeviceProtection_Yes                     4930 non-null   float64
 10  TechSupport_Yes                          4930

## 5. TREINAMENTO E PREDIÇÃO DO MODELO

In [84]:
# TREINANDO TODOS OS MODELOS
regLog = LogisticRegression(C=1,max_iter=1000).fit(X_train_final, y_train)
rf = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced').fit(X_train_final, y_train)
knn = KNeighborsClassifier(n_neighbors=12, metric='euclidean').fit(X_train_final, y_train)
xgb_classifier = XGBClassifier(n_estimators=100,max_depth=3).fit(X_train_final, y_train)

# PREVENDO O RESULTADO DOS MODELOS
yLog = regLog.predict(X_test_final)
yrf = rf.predict(X_test_final)
yknn = knn.predict(X_test_final)
yxgb = xgb_classifier.predict(X_test_final)

In [85]:
# ANALISANDO A ACURÁCIA DOS MODELOS E SELECIONANDO O MELHOR
print(f"Accuracy LogisticRegression: {accuracy_score(yLog,y_test):.3f}" )
print(f"Accuracy RandomForestClassifier: {accuracy_score(yrf,y_test):.3f}" )
print(f"Accuracy KNeighborsClassifier: {accuracy_score(yknn,y_test):.3f}" )
print(f"Accuracy XGBClassifier: {accuracy_score(yxgb,y_test):.3f}" )

Accuracy LogisticRegression: 0.783
Accuracy RandomForestClassifier: 0.780
Accuracy KNeighborsClassifier: 0.776
Accuracy XGBClassifier: 0.787
